In [150]:
import pandas as pd
import numpy as np

In [151]:
import matplotlib.pyplot as plt

In [152]:
df_India=pd.read_csv('CSV/India_timeline.csv',index_col='Date',parse_dates=True)

In [4]:
len(df_India)

476

In [5]:
df_India.drop(['Unnamed: 0'],axis=1,inplace=True)

In [6]:
df_India.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 476 entries, 2020-01-30 to 2021-05-19
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Confirmed       476 non-null    float64
 1   Deaths          476 non-null    float64
 2   Cured           476 non-null    float64
 3   Daily_Change_C  476 non-null    int64  
 4   Daily_Change_R  476 non-null    int64  
 5   Daily_Change_D  476 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 26.0 KB


In [7]:
df_world=pd.read_csv('CSV/worldwide_daily_change.csv',index_col='index',parse_dates=True)
df_world.index.freq = 'D'

In [8]:
df_world.drop('Unnamed: 0',axis=1,inplace=True)

In [9]:
df_world.tail()

,Daily_Change_C,Daily_change_R,Daily_change_D
index,,,
2021-05-19,671092,698027,13344
2021-05-20,628529,641259,12830
2021-05-21,607596,613571,12598
2021-05-22,560116,605485,10809
2021-05-23,478141,509894,9813


In [10]:
len(df_world)

488

In [11]:
df_world

,Daily_Change_C,Daily_change_R,Daily_change_D
index,,,
2020-01-22,0,0,0
2020-01-23,98,2,1
2020-01-24,286,7,8
2020-01-25,492,3,16
2020-01-26,685,14,14
...,...,...,...
2021-05-19,671092,698027,13344
2021-05-20,628529,641259,12830
2021-05-21,607596,613571,12598


In [12]:
pwd

'/Users/ganji/Desktop/ADQ/Case_study/covid_poc'

In [13]:

from flask import Flask, request, render_template, jsonify # Import flask libraries
import pickle

In [213]:
app = Flask(__name__)
model_India_sarimax = pickle.load(open('models/model_India_SARIMAX.pkl', 'rb'))
# model_India_AR_1=pickle.load(open('models/model_India_AR_1.pkl', 'rb'))
model_India_AR_37=pickle.load(open('models/model_India_AR_37.pkl','rb'))
# models_India=['model_India_sarimax','model_India_AR_1','model_India_AR_37']
model_world_sarimax = pickle.load(open('models/model_world_SARIMAX.pkl', 'rb'))
# model_world_AR_1=pickle.load(open('models/model_world_AR_1.pkl', 'rb'))
model_world_AR_18=pickle.load(open('models/model_world_AR_18.pkl','rb'))
models=['model_India_sarimax','model_world_sarimax','model_India_AR_37','model_world_AR_18']
# models_world=['model_world_sarimax','model_world_AR_1','model_world_AR_18']
# model_world = pickle.load(open('models/model_world.pkl', 'rb'))
headings=("Date","Cases")
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')
@app.route('/result', methods=['GET','POST'])
def predict():
    days=request.args.get('noOfDays')
#     value=request.args.get('data')
    days=int(days)
#     if value=='India':
    start=len(df_India)
    stop=len(df_India)+days
    start_w=len(df_world)
    stop_w=len(df_world)+days
    for i in models:
        if i=='model_India_sarimax':

            result=model_India_sarimax.predict(start,stop-1,dynamic=False).rename('forecast')
            result=pd.Series(result).astype(int)
            plt.plot(df_India['Daily_Change_C'],label="Daily confirmed cases")
            
            result.plot(legend=True,figsize=(12,6))
            plt.title("Confirmed covid cases forecasting")
            plt.legend(loc=(0.01, 0.85))
            
            
            plt.savefig('static/SARIMAX_India.png')
            plt.close()
            result=result.to_frame()
            result.reset_index(inplace=True)
            result['index']=pd.to_datetime(result['index']).dt.strftime('%Y-%b-%d')
#             print(result.columns)
            result['forecast']=result['forecast'].apply('{:,}'.format)
#             print(result)
#                 result.to_csv('sarimax_india.csv')
            result = list(result.values)
        
        elif i=='model_world_sarimax':
            result1=model_world_sarimax.predict(start_w,stop_w,dynamic=False).rename('forecast')
            
            result1=pd.Series(result1).astype(int)
            result1=np.around(result1)
            plt.plot(df_world['Daily_Change_C'],label="Daily confirmed cases")
            result1.plot(legend=True,figsize=(12,6))
            plt.title("Confirmed covid cases forecasting")
            
            
            plt.legend(loc=(0.01, 0.85))
            
            plt.savefig('static/SARIMAX_world.png')
            plt.close()
            result1=result1.to_frame()
            result1.reset_index(inplace=True)
            result1['index']=pd.to_datetime(result1['index']).dt.strftime('%d-%b-%Y')
            result1['forecast']=result1['forecast'].apply('{:,}'.format)
#                 result.to_csv('sarimax_world.csv')
            result1 = list(result1.values)


#         elif i=='model_India_AR_1':
#             result2=model_India_AR_1.predict(start,stop,dynamic=False).rename('forecast')
#             result2=pd.Series(result2).astype(int)
#             df_India['Daily_Change_C'].plot(legend=True)
#             result2.plot(legend=True,figsize=(12,6))
#             plt.savefig('static/AR_1_India.png')
#             plt.close()
#             result2=result2.to_frame()
#             result2.reset_index(inplace=True)
#             result2['index'] = pd.to_datetime(result2['index']).dt.date
# #                 result1.to_csv('AR_1_india.csv')
#             result2 = list(result2.values)
    
#         elif i=='model_world_AR_1':
#             result3=model_world_AR_1.predict(start_w,stop_w,dynamic=False).rename('forecast')
#             result3=pd.Series(result3).astype(int)
#             result3=np.around(result3)
#             df_world['Daily_Change_C'].plot(legend=True)
#             result3.plot(legend=True,figsize=(12,6))
#             plt.savefig('static/AR_1_world.png')
#             plt.close()
#             result3=result3.to_frame()
#             result3.reset_index(inplace=True)
#             result3['index'] = pd.to_datetime(result3['index']).dt.date
# #                 result1.to_csv('AR_1_world.csv')
#             result3 = list(result3.values)
    
        elif i=='model_India_AR_37':
            result4=model_India_AR_37.predict(start,stop-1,dynamic=False).rename('forecast')
            result4=pd.Series(result4).astype(int)
            plt.plot(df_India['Daily_Change_C'],label="Daily confirmed cases")
            result4.plot(legend=True,figsize=(12,6))
            
            plt.title("Confirmed covid cases forecasting")
            plt.legend(loc=(0.01, 0.85))
            
            plt.savefig('static/AR_37_India.png')
            plt.close()
            result4=result4.to_frame()
            result4.reset_index(inplace=True)
            result4['index']=pd.to_datetime(result4['index']).dt.strftime('%d-%b-%Y')
            result4['forecast']=result4['forecast'].apply('{:,}'.format)
#                 result2.to_csv('AR_37_india.csv')
            result4 = list(result4.values)
    
        elif i=='model_world_AR_18':
            result5=model_world_AR_18.predict(start_w,stop_w-1,dynamic=False).rename('forecast')
            result5=pd.Series(result5).astype(int)
            result5=np.around(result5)
            plt.plot(df_world['Daily_Change_C'],label="Daily confirmed cases")
            result5.plot(legend=True,figsize=(12,6))
            plt.title("Confirmed covid cases forecasting")           
            plt.legend(loc=(0.01, 0.85))           
            plt.savefig('static/AR_18_world.png')
            plt.close()
            result5=result5.to_frame()
            result5.reset_index(inplace=True)
            result5['index']=pd.to_datetime(result5['index']).dt.strftime('%d-%b-%Y')
            result5['forecast']=result5['forecast'].apply('{:,}'.format)
#                 result2.to_csv('AR_37_world.csv')
            result5 = list(result5.values)
        result6=pd.read_csv('static/lstm_india.csv')
        result6['Cases']=result6['Cases'].astype(int)
        result6['Date']=pd.to_datetime(result6['Date']).dt.strftime('%d-%b-%Y')
        result6['Cases']=result6['Cases'].apply('{:,}'.format)
        result6 = list(result6.values)
        result7=pd.read_csv('static/lstm_world.csv')
        result7['Cases']=result7['Cases'].astype(int)
        result7['Date']=pd.to_datetime(result7['Date']).dt.strftime('%d-%b-%Y')
        result7['Cases']=result7['Cases'].apply('{:,}'.format)
        result7 = list(result7.values)
    return render_template('result.html',headings=headings,data=result4,data1=result5,data4=result,data5=result1,data6=result6,data7=result7)
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jun/2021 20:47:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:54] "GET /result?noOfDays=5 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/js/index.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/plotly_india.png HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/AR_37_India.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/plotly_world.png HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/SARIMAX_India.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/AR_18_world.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/SARIMAX_world.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/model_india_lstm.png HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2021 20:47:55] "GET /static/model_wor

In [84]:
result3=pd.read_csv('static/lstm_india.csv')

In [21]:
result3

,Date,Cases
0,2021-05-20,367954.0
1,2021-05-21,351169.0
2,2021-05-22,339105.0
3,2021-05-23,337320.0
4,2021-05-24,330572.0


In [131]:
import datetime

In [134]:
result3['Date']=pd.to_datetime(result3['Date']).dt.strftime('%Y-%b-%d')

In [148]:
result3['Cases']=result3.Cases.apply(lambda x : "{:,}".format(x))

In [147]:
result3['']

,Date,Cases
0,2021-May-20,367954.0
1,2021-May-21,351169.0
2,2021-May-22,339105.0
3,2021-May-23,337320.0
4,2021-May-24,330572.0


In [137]:
result6=pd.read_csv('static/lstm_india.csv')

In [138]:
result6

,Date,Cases
0,2021-05-20,367954.0
1,2021-05-21,351169.0
2,2021-05-22,339105.0
3,2021-05-23,337320.0
4,2021-05-24,330572.0
